In [1]:
from common_import import *
result_dict = readpkl('result_dict.pkl')

In [2]:
def print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=True):
    import prettytable

    # PrettyTable 객체 생성 및 필드 설정
    pt = prettytable.PrettyTable()
    pt.field_names = ['Dataset', 'E1', 'E2', 'Checkpoint', 'Subset']


    # 데이터를 리스트에 수집
    data = []
    for key in checkpoint_list:
        if key not in result_dict.keys():
            print(f"Checkpoint {key}에 대한 결과가 없습니다.")
            continue
        for subset in result_dict[key].keys():
            #if 'WITH_RZ' in subset: continue
            if 'STEP' in subset: continue
            if 'TRAIN_UNIV' in subset: continue
            splited = subset.split('-')
            if 'CAM_NO_FACTOR' in splited[-1] or 'CANONICAL' in splited[-1]:
                dataset = splited[0]
            else:
                dataset = splited[0] + '-' + splited[-1]
            #print(dataset, type(dataset))
            e1 = float(result_dict[key][subset]['e1'])
            e2 = float(result_dict[key][subset]['e2'])
            data.append([dataset, e1, e2, key, subset])

    # 데이터를 Subset 1순위, checkpoint 2순위로 정렬
    data_sorted = sorted(data, key=lambda x: (x[0], x[3]))

    prev_dataset = ''
    for i, row in enumerate(data_sorted):
        if prev_dataset != '' and prev_dataset != row[0]:
            data_sorted[i-1].append(True)
        else:
            data_sorted[i-1].append(False)
        prev_dataset = row[0]

    # 정렬된 데이터를 테이블에 추가
    for row in data_sorted:
        pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', row[3], row[4]], divider=row[-1])

    # 테이블 출력
    print(pt)

    # 엑셀로 저장
    if save_excel:
        df = pd.DataFrame(pt.rows, columns=pt.field_names)
        df.to_excel(output_file, index=False)
        print(f"PrettyTable 데이터를 {output_file}로 저장했습니다!")

In [5]:
checkpoint_list = [
    "MB_train_h36m_gt_cam_no_factor_with_canonical2",
    'MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1_ts_s5678',
    'MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03',
    'MB_train_fit3d_gt_cam_no_factor_with_canonical2_ts_s4710',
]
output_file = f"output_with_canonical2.xlsx"
print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=False)
print()

+---------------------+--------+--------+---------------------------------------------------------------+--------------------------------------+
|       Dataset       |   E1   |   E2   |                           Checkpoint                          |                Subset                |
+---------------------+--------+--------+---------------------------------------------------------------+--------------------------------------+
| 3DHP-TEST_ALL_TRAIN | 186.13 | 125.26 |     MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03    | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 171.22 | 108.59 |    MB_train_fit3d_gt_cam_no_factor_with_canonical2_ts_s4710   | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 115.10 | 57.43  |         MB_train_h36m_gt_cam_no_factor_with_canonical2        | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 122.79 | 66.97  | MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1_ts_s5678 | 3DHP-GT-CAM_NO_FACTOR-TE

# H36M S15678 TRAIN

In [4]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering{seed}',
        #f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering{seed}',
    ]
    output_file = f"output_h36m{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=False)
    print()

+----------------------+--------+--------+---------------------------------------------------------------------------------+-------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                    Checkpoint                                   |                                  Subset                                 |
+----------------------+--------+--------+---------------------------------------------------------------------------------+-------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 95.59  | 68.06  |                          MB_train_h36m_gt_cam_no_factor                         |                   3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TEST_ALL_TRAIN  | 70.31  | 56.37  | MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering |  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN  |
| 3DH

# H36M input centering

In [3]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor_input_centering{seed}',
    ]
    output_file = f"output_h36m{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=False)
    print()

+----------------------+--------+-------+------------------------------------------------+---------------------------------------+
|       Dataset        |   E1   |   E2  |                   Checkpoint                   |                 Subset                |
+----------------------+--------+-------+------------------------------------------------+---------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 115.18 | 58.73 | MB_train_h36m_gt_cam_no_factor_input_centering |  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
+----------------------+--------+-------+------------------------------------------------+---------------------------------------+
|   3DHP-TEST_TS1_6    | 107.49 | 55.61 | MB_train_h36m_gt_cam_no_factor_input_centering |    3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6   |
+----------------------+--------+-------+------------------------------------------------+---------------------------------------+
| 3DHP-TEST_TS1_6_UNIV | 89.16  | 49.59 | MB_train_h36m_gt_cam_no_factor_input_cent

# H36M with Rz

In [4]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering{seed}',
    ]
    output_file = f"output_h36m{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=False)
    print()

+----------------------+-------+-------+---------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset        |   E1  |   E2  |                                    Checkpoint                                   |                                 Subset                                 |
+----------------------+-------+-------+---------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 81.27 | 61.96 | MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering | 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN  |
+----------------------+-------+-------+---------------------------------------------------------------------------------+------------------------------------------------------------------------+
|   3DHP-TEST_TS1_6 

In [ ]:
import prettytable

checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering{seed}',
    ]
print(checkpoint_list)

# 데이터를 리스트에 수집
data = {}
for key in checkpoint_list:
    if key not in result_dict.keys():
        print(f"Checkpoint {key}에 대한 결과가 없습니다.")
        continue
    if 'seed1' in key: seed = 1
    elif 'seed2' in key: seed = 2
    elif 'seed3' in key: seed = 3
    elif 'seed4' in key: seed = 4
    else: seed = 0
    for subset in result_dict[key].keys():
        if subset not in data.keys():
            data[subset] = {}
        data[subset][seed] = {}
        data[subset][seed]['e1'] = float(result_dict[key][subset]['e1'])
        data[subset][seed]['e2'] = float(result_dict[key][subset]['e2'])

# PrettyTable 객체 생성 및 필드 설정
pt = prettytable.PrettyTable()
pt.field_names = ['Subset', 'seed0_e1', 'seed1_e1', 'seed2_e1', 'seed3_e1', 'seed4_e1', 'Average_e1', 'seed0_e2', 'seed1_e2', 'seed2_e2', 'seed3_e2', 'seed4_e2', 'Average_e2']

rows = []
for subset in data.keys():
    rows.append([subset+'_e1', data[subset][0]['e1'], data[subset][1]['e1'], data[subset][2]['e1'], data[subset][3]['e1'], data[subset][4]['e1'], np.mean([data[subset][i]['e1'] for i in range(5)]), data[subset][0]['e2'], data[subset][1]['e2'], data[subset][2]['e2'], data[subset][3]['e2'], data[subset][4]['e2'], np.mean([data[subset][i]['e2'] for i in range(5)])])

rows = sorted(rows, key=lambda x: (x[0]))

# 정렬된 데이터를 테이블에 추가
for row in rows:
    pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', f'{row[3]:.2f}', f'{row[4]:.2f}', f'{row[5]:.2f}', f'{row[6]:.2f}', f'{row[7]:.2f}', f'{row[8]:.2f}', f'{row[9]:.2f}', f'{row[10]:.2f}', f'{row[11]:.2f}', f'{row[12]:.2f}'])
print(pt)

# save to excel
# df = pd.DataFrame(pt.rows, columns=pt.field_names)
# df.to_excel('output_h36m_revolute_input_centering.xlsx', index=False)

['MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_seed1', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_seed2', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_seed4']


KeyError: 3

# H36M no Rz

In [19]:
import prettytable

checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering{seed}',
    ]
print(checkpoint_list)

# 데이터를 리스트에 수집
data = {}
for key in checkpoint_list:
    if key not in result_dict.keys():
        print(f"Checkpoint {key}에 대한 결과가 없습니다.")
        continue
    if 'seed1' in key: seed = 1
    elif 'seed2' in key: seed = 2
    elif 'seed3' in key: seed = 3
    elif 'seed4' in key: seed = 4
    else: seed = 0
    for subset in result_dict[key].keys():
        if 'CANONICAL_PCL' in subset: continue
        if subset not in data.keys():
            data[subset] = {}
        data[subset][seed] = {}
        data[subset][seed]['e1'] = float(result_dict[key][subset]['e1'])
        data[subset][seed]['e2'] = float(result_dict[key][subset]['e2'])

# PrettyTable 객체 생성 및 필드 설정
pt = prettytable.PrettyTable()
pt.field_names = ['Subset', 'seed0_e1', 'seed1_e1', 'seed2_e1', 'seed3_e1', 'seed4_e1', 'Average_e1', 'seed0_e2', 'seed1_e2', 'seed2_e2', 'seed3_e2', 'seed4_e2', 'Average_e2']

rows = []
for subset in data.keys():
    rows.append([subset+'_e1', data[subset][0]['e1'], data[subset][1]['e1'], data[subset][2]['e1'], data[subset][3]['e1'], data[subset][4]['e1'], np.mean([data[subset][i]['e1'] for i in range(5)]), data[subset][0]['e2'], data[subset][1]['e2'], data[subset][2]['e2'], data[subset][3]['e2'], data[subset][4]['e2'], np.mean([data[subset][i]['e2'] for i in range(5)])])

rows = sorted(rows, key=lambda x: (x[0]))

# 정렬된 데이터를 테이블에 추가
for row in rows:
    pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', f'{row[3]:.2f}', f'{row[4]:.2f}', f'{row[5]:.2f}', f'{row[6]:.2f}', f'{row[7]:.2f}', f'{row[8]:.2f}', f'{row[9]:.2f}', f'{row[10]:.2f}', f'{row[11]:.2f}', f'{row[12]:.2f}'])
print(pt)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36m_revolute_no_Rz_input_centering.xlsx', index=False)

['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_seed1', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_seed2', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_seed3', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_seed4']
+---------------------------------------------------------------------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|                                      Subset                                     | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+---------------------------------------------------------------------------------+-------

In [20]:
import prettytable

checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering{seed}',
    ]
print(checkpoint_list)

# 데이터를 리스트에 수집
data = {}
for key in checkpoint_list:
    if key not in result_dict.keys():
        print(f"Checkpoint {key}에 대한 결과가 없습니다.")
        continue
    if 'seed1' in key: seed = 1
    elif 'seed2' in key: seed = 2
    elif 'seed3' in key: seed = 3
    elif 'seed4' in key: seed = 4
    else: seed = 0
    for subset in result_dict[key].keys():
        if 'CANONICAL_PCL' in subset: continue
        if subset not in data.keys():
            data[subset] = {}
        data[subset][seed] = {}
        data[subset][seed]['e1'] = float(result_dict[key][subset]['e1'])
        data[subset][seed]['e2'] = float(result_dict[key][subset]['e2'])

# PrettyTable 객체 생성 및 필드 설정
pt = prettytable.PrettyTable()
pt.field_names = ['Subset', 'seed0_e1', 'seed1_e1', 'seed2_e1', 'seed3_e1', 'seed4_e1', 'Average_e1', 'seed0_e2', 'seed1_e2', 'seed2_e2', 'seed3_e2', 'seed4_e2', 'Average_e2']

rows = []
for subset in data.keys():
    rows.append([subset+'_e1', data[subset][0]['e1'], data[subset][1]['e1'], data[subset][2]['e1'], data[subset][3]['e1'], data[subset][4]['e1'], np.mean([data[subset][i]['e1'] for i in range(5)]), data[subset][0]['e2'], data[subset][1]['e2'], data[subset][2]['e2'], data[subset][3]['e2'], data[subset][4]['e2'], np.mean([data[subset][i]['e2'] for i in range(5)])])

rows = sorted(rows, key=lambda x: (x[0]))

# 정렬된 데이터를 테이블에 추가
for row in rows:
    pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', f'{row[3]:.2f}', f'{row[4]:.2f}', f'{row[5]:.2f}', f'{row[6]:.2f}', f'{row[7]:.2f}', f'{row[8]:.2f}', f'{row[9]:.2f}', f'{row[10]:.2f}', f'{row[11]:.2f}', f'{row[12]:.2f}'])
print(pt)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36m_revolute_input_centering.xlsx', index=False)

['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed1', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed2', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed3', 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed4']
+---------------------------------------------------------------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|                                   Subset                                  | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+---------------------------------------------------------------------------+----------+----------+----------+----------+----------+

# H36M S1 TRAIN

In [6]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_ts_s5678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678{seed}',
    ]
    output_file = f"output_h36m_tr_s1_ts_s5678{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=False)
    print()

+----------------------+--------+--------+------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                              Checkpoint                                              |                                    Subset                                    |
+----------------------+--------+--------+------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 86.53  | 65.30  |    MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678    |    3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN     |
| 3DHP-TEST_ALL_TRAIN  | 86.36  | 64.76  | MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_re

# FIT3D TR_S03

In [3]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_fit3d_gt_cam_no_factor_tr_s03{seed}',
        f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03{seed}',
        f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03{seed}',
    ]
    print(f"**** seed = {seed} ****")
    output_file = f"output_fit3d_tr_s03{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

**** seed =  ****
+----------------------+--------+--------+-----------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                        Checkpoint                                       |                                  Subset                                 |
+----------------------+--------+--------+-----------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 178.32 | 128.28 | MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03 |  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN  |
| 3DHP-TEST_ALL_TRAIN  | 152.04 | 113.19 |         MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03        |  3DHP-GT-CAM_NO_FACTO

# FIT3D TS_S4710

In [4]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_fit3d_gt_cam_no_factor_ts_s4710{seed}',
        f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710{seed}',
        f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710{seed}',
    ]
    output_file = f"output_fit3d_ts_s4710{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

+----------------------+--------+--------+-------------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                         Checkpoint                                        |                                  Subset                                 |
+----------------------+--------+--------+-------------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 165.81 | 114.84 | MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710 |    3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_REVOLUTE-TEST_ALL_TRAIN   |
| 3DHP-TEST_ALL_TRAIN  | 159.60 | 116.58 |         MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710        |  3DHP-GT-CAM_NO_FACTOR-INPUT_

# 3DHP S1

In [5]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_3dhp_gt_cam_no_factor_tr_s1{seed}',
        f'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1{seed}',
        f'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1{seed}',
    ]
    print(f"**** seed = {seed} ****")
    output_file = f"output_3dhp_tr_s1{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

**** seed =  ****
+----------------------+--------+--------+---------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                       Checkpoint                                      |                                  Subset                                 |
+----------------------+--------+--------+---------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
|   3DHP-TEST_TS1_6    | 66.44  | 47.83  | MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1 |      3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_REVOLUTE-TEST_TS1_6     |
|   3DHP-TEST_TS1_6    | 66.28  | 45.22  |         MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1        |    3DHP-GT-CAM_NO_FACTOR-INPUT_

# Total

In [ ]:
import prettytable
summary_table = prettytable.PrettyTable()
summary_table.field_names = ['checkpoint', 'subset', 'MPJPE', 'P-MPJPE']
for checkpoint in result_dict.keys():
    for subset in result_dict[checkpoint].keys():
        e1 = result_dict[checkpoint][subset]['e1']
        e2 = result_dict[checkpoint][subset]['e2']
        row = [checkpoint, subset, f'{e1:.2f}', f'{e2:.2f}']
        summary_table.add_row(row)
print(summary_table)

+-------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+--------+---------+
|                                                                       checkpoint                                                                      |                                          subset                                          | MPJPE  | P-MPJPE |
+-------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+--------+---------+
|                        MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1                       |     H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXE